In [1]:
!pip install ultralytics==8.0.20
!pip install deepface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=5f84ba10fb8d2e8136767a35d2d9d9be086365a10837d664a93d4b418e2bee1d
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
from google.colab.patches import cv2_imshow
import pickle  # For saving embeddings and labels

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Dictionary to store embeddings and labels
face_database = {}

# Path to the folder containing images
folder_path = '/content/drive/MyDrive/Test_Data/Train'  # Replace with the path to your folder

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        image_path = os.path.join(folder_path, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Extract the label from the image filename (e.g., "abc.jpg" -> "abc")
        label = os.path.splitext(filename)[0]

        # Predict faces using the trained YOLO model
        results = model.predict(image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = image[int(y1):int(y2), int(x1):int(x2)]

            # Optionally, resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Get the embedding for the cropped face
            embedding = get_face_embedding(face_image)

            # Add the embedding and label to the face database
            face_database[label] = embedding

            # Optionally, print or save the embeddings with labels
            print(f"Processed {label}: Embedding added.")

            # Draw bounding box and label on the image
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the processed image
        #cv2_imshow(image)

# Save the face database for future use
with open("face_database.pkl", "wb") as file:
    pickle.dump(face_database, file)

print("Face database saved successfully!")


25-01-07 14:19:44 - Directory /root/.deepface has been created
25-01-07 14:19:44 - Directory /root/.deepface/weights has been created


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded


25-01-07 14:19:53 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:02<00:00, 31.5MB/s]


Processed Sushma: Embedding added.
Processed Karthik: Embedding added.
Processed Prasad: Embedding added.
Processed Gayatri: Embedding added.
Processed Anusha: Embedding added.
Processed Swaroop: Embedding added.
Face database saved successfully!


In [3]:
import pickle

# Load the pickle file
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

# Check the number of entries
print(f"Number of entries in the pickle file: {len(face_database)}")

# Optional: Print the keys (labels) to inspect the data
print("Labels in the pickle file:", list(face_database.keys()))


Number of entries in the pickle file: 6
Labels in the pickle file: ['Sushma', 'Karthik', 'Prasad', 'Gayatri', 'Anusha', 'Swaroop']


In [4]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Load the saved face database
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Threshold for cosine similarity (tune based on your use case)
SIMILARITY_THRESHOLD = 0.5

# Path to the folder containing test images
test_folder_path = '/content/drive/MyDrive/Test_Data/Test'  # Replace with your folder path

# Loop through all images in the folder
for filename in os.listdir(test_folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        test_image_path = os.path.join(test_folder_path, filename)

        # Read the test image
        test_image = cv2.imread(test_image_path)

        # Predict faces using the trained YOLO model
        results = model.predict(test_image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = test_image[int(y1):int(y2), int(x1):int(x2)]

            # Resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Get the embedding for the cropped face
            new_embedding = get_face_embedding(face_image)

            # Initialize variables to track the best match
            best_label = "Unknown"
            best_similarity = 0

            # Compare the new embedding with stored embeddings in the face database
            for label, stored_embedding in face_database.items():
                similarity = cosine_similarity([new_embedding], [stored_embedding])[0][0]
                if similarity > best_similarity and similarity > SIMILARITY_THRESHOLD:
                    best_label = label
                    best_similarity = similarity

            # Draw bounding box and label on the image
            cv2.rectangle(test_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(test_image, best_label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print(f"Image: {filename}, Detected face is labeled as: {best_label} (similarity: {best_similarity:.2f})")

        # Save or display the image with labels
        output_image_path = os.path.join('/content/verified_images', filename)  # Adjust the output folder as needed
        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
        cv2.imwrite(output_image_path, test_image)

print("Face recognition completed for all images in the folder.")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Image: Sushma.jpg, Detected face is labeled as: Sushma (similarity: 0.71)
Image: Gayatri.jpg, Detected face is labeled as: Gayatri (similarity: 0.83)
Image: Anusha.jpg, Detected face is labeled as: Anusha (similarity: 0.82)
Image: Group.jpg, Detected face is labeled as: Prasad (similarity: 0.70)
Image: Group.jpg, Detected face is labeled as: Swaroop (similarity: 0.87)
Image: Group.jpg, Detected face is labeled as: Karthik (similarity: 0.72)
Image: Prasad.jpeg, Detected face is labeled as: Prasad (similarity: 0.63)
Image: Swaroop.jpg, Detected face is labeled as: Swaroop (similarity: 1.00)
Image: frame_0051.jpg, Detected face is labeled as: Unknown (similarity: 0.00)
Image: frame_0055.jpg, Detected face is labeled as: Unknown (similarity: 0.00)
Image: frame_0060.jpg, Detected face is labeled as: Unknown (similarity: 0.00)
Image: frame_0077.jpg, Detected face is labeled as: Prasad (similarity: 0.70)
Face recognition completed for all images in the folder.


In [5]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import pickle  # For saving embeddings and labels
import albumentations as A
from albumentations.core.composition import OneOf

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Function to apply augmentations to an image
def augment_image(image):
    augmentations = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.GaussianBlur(p=0.3),
        A.HueSaturationValue(p=0.5),
    ])
    augmented = augmentations(image=image)
    return augmented['image']

# Dictionary to store embeddings and labels
face_database = {}

# Path to the folder containing input images
input_folder = '/content/drive/MyDrive/Test_Data/Train'  # Replace with the path to your input folder

# Path to the folder to save processed images
output_folder = '/content/processed_images'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# Loop through all images in the folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        image_path = os.path.join(input_folder, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Extract the label from the image filename (e.g., "abc.jpg" -> "abc")
        label = os.path.splitext(filename)[0]

        # Predict faces using the trained YOLO model
        results = model.predict(image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = image[int(y1):int(y2), int(x1):int(x2)]

            # Optionally, resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Generate multiple embeddings using augmentations
            embeddings = []
            for _ in range(5):  # Generate 5 augmented versions
                augmented_face = augment_image(face_image)
                embedding = get_face_embedding(augmented_face)
                embeddings.append(embedding)

            # Add all embeddings to the face database under the same label
            if label not in face_database:
                face_database[label] = []
            face_database[label].extend(embeddings)

            # Draw bounding box and label on the image
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the processed image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)
        print(f"Saved processed image: {output_path}")

# Save the face database for future use
with open("face_database.pkl", "wb") as file:
    pickle.dump(face_database, file)

print("Face database saved successfully!")

# Testing with new images folder
new_images_folder = '/content/drive/MyDrive/Test_Data/Test'  # Replace with the path to your folder containing new images
output_test_folder = '/content/processed_test_images'
os.makedirs(output_test_folder, exist_ok=True)  # Create the folder for test outputs if it doesn't exist

for filename in os.listdir(new_images_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        new_image_path = os.path.join(new_images_folder, filename)
        new_image = cv2.imread(new_image_path)

        # Predict faces using the trained YOLO model
        results = model.predict(new_image_path)

        for result in results[0].boxes.data:
            x1, y1, x2, y2 = result[:4]
            face_image = new_image[int(y1):int(y2), int(x1):int(x2)]
            face_image = cv2.resize(face_image, (160, 160))

            # Generate embedding for the detected face
            embedding = get_face_embedding(face_image)

            # Find the closest match in the database
            best_match = None
            best_similarity = -1
            for label, embeddings in face_database.items():
                for stored_embedding in embeddings:
                    similarity = np.dot(embedding, stored_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(stored_embedding))
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match = label

            # Draw results on the image
            match_label = best_match if best_similarity > 0.5 else "Unknown"
            cv2.rectangle(new_image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(new_image, match_label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            print(f"Image: {filename}, Detected face is labeled as: {best_match} (similarity: {best_similarity:.2f})")

        # Save the processed test image to the output folder
        output_test_path = os.path.join(output_test_folder, filename)
        cv2.imwrite(output_test_path, new_image)
        #print(f"Test image saved with labels: {output_test_path}")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Saved processed image: /content/processed_images/Sushma.jpg
Saved processed image: /content/processed_images/Karthik.jpg
Saved processed image: /content/processed_images/Prasad.jpg
Saved processed image: /content/processed_images/Gayatri.jpg
Saved processed image: /content/processed_images/Anusha.jpg
Saved processed image: /content/processed_images/Swaroop.jpg
Face database saved successfully!
Image: Sushma.jpg, Detected face is labeled as: Sushma (similarity: 0.71)
Image: Gayatri.jpg, Detected face is labeled as: Gayatri (similarity: 0.86)
Image: Anusha.jpg, Detected face is labeled as: Anusha (similarity: 0.82)
Image: Group.jpg, Detected face is labeled as: Prasad (similarity: 0.73)
Image: Group.jpg, Detected face is labeled as: Swaroop (similarity: 0.85)
Image: Group.jpg, Detected face is labeled as: Karthik (similarity: 0.74)
Image: Prasad.jpeg, Detected face is labeled as: Prasad (similarity: 0.69)
Image: Swaroop.jpg, Detected face is labeled as: Swaroop (similarity: 0.97)
Image: 

In [6]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import pickle  # For saving embeddings and labels
import albumentations as A
from albumentations.core.composition import OneOf

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Function to apply augmentations to an image
def augment_image(image):
    augmentations = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.GaussianBlur(p=0.3),
        A.HueSaturationValue(p=0.5),
    ])
    augmented = augmentations(image=image)
    return augmented['image']

# Convert image to grayscale
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Dictionary to store embeddings and labels
face_database = {}

# Path to the folder containing input images
input_folder = '/content/drive/MyDrive/Test_Data/Train'  # Replace with the path to your input folder

# Path to the folder to save processed images
output_folder = '/content/processed_images_Gray'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# Loop through all images in the folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        image_path = os.path.join(input_folder, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Convert image to grayscale
        grayscale_image = convert_to_grayscale(image)
        grayscale_image = cv2.cvtColor(grayscale_image, cv2.COLOR_GRAY2BGR)  # Convert back to 3 channels for DeepFace

        # Extract the label from the image filename (e.g., "abc.jpg" -> "abc")
        label = os.path.splitext(filename)[0]

        # Predict faces using the trained YOLO model
        results = model.predict(image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = grayscale_image[int(y1):int(y2), int(x1):int(x2)]

            # Optionally, resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Generate multiple embeddings using augmentations
            embeddings = []
            for _ in range(5):  # Generate 5 augmented versions
                augmented_face = augment_image(face_image)
                embedding = get_face_embedding(augmented_face)
                embeddings.append(embedding)

            # Add all embeddings to the face database under the same label
            if label not in face_database:
                face_database[label] = []
            face_database[label].extend(embeddings)

            # Draw bounding box and label on the image
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the processed image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)
        print(f"Saved processed image: {output_path}")

# Save the face database for future use
with open("face_database.pkl", "wb") as file:
    pickle.dump(face_database, file)

print("Face database saved successfully!")

# Testing with new images folder
new_images_folder = '/content/drive/MyDrive/Test_Data/Test'  # Replace with the path to your folder containing new images
output_test_folder = '/content/processed_test_images_Gray'
os.makedirs(output_test_folder, exist_ok=True)  # Create the folder for test outputs if it doesn't exist

for filename in os.listdir(new_images_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        new_image_path = os.path.join(new_images_folder, filename)
        new_image = cv2.imread(new_image_path)

        # Convert new image to grayscale
        grayscale_image = convert_to_grayscale(new_image)
        grayscale_image = cv2.cvtColor(grayscale_image, cv2.COLOR_GRAY2BGR)  # Convert back to 3 channels for DeepFace

        # Predict faces using the trained YOLO model
        results = model.predict(new_image_path)

        for result in results[0].boxes.data:
            x1, y1, x2, y2 = result[:4]
            face_image = grayscale_image[int(y1):int(y2), int(x1):int(x2)]
            face_image = cv2.resize(face_image, (160, 160))

            # Generate embedding for the detected face
            embedding = get_face_embedding(face_image)

            # Find the closest match in the database
            best_match = None
            best_similarity = -1
            for label, embeddings in face_database.items():
                for stored_embedding in embeddings:
                    similarity = np.dot(embedding, stored_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(stored_embedding))
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match = label

            # Draw results on the image
            match_label = best_match if best_similarity > 0.5 else "Unknown"
            cv2.rectangle(new_image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(new_image, match_label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            print(f"Image: {filename}, Detected face is labeled as: {best_match} (similarity: {best_similarity:.2f})")

        # Save the processed test image to the output folder
        output_test_path = os.path.join(output_test_folder, filename)
        cv2.imwrite(output_test_path, new_image)
        #print(f"Test image saved with labels: {output_test_path}")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Saved processed image: /content/processed_images_Gray/Sushma.jpg
Saved processed image: /content/processed_images_Gray/Karthik.jpg
Saved processed image: /content/processed_images_Gray/Prasad.jpg
Saved processed image: /content/processed_images_Gray/Gayatri.jpg
Saved processed image: /content/processed_images_Gray/Anusha.jpg
Saved processed image: /content/processed_images_Gray/Swaroop.jpg
Face database saved successfully!
Image: Sushma.jpg, Detected face is labeled as: Sushma (similarity: 0.73)
Image: Gayatri.jpg, Detected face is labeled as: Gayatri (similarity: 0.84)
Image: Anusha.jpg, Detected face is labeled as: Anusha (similarity: 0.81)
Image: Group.jpg, Detected face is labeled as: Prasad (similarity: 0.81)
Image: Group.jpg, Detected face is labeled as: Swaroop (similarity: 0.86)
Image: Group.jpg, Detected face is labeled as: Karthik (similarity: 0.80)
Image: Prasad.jpeg, Detected face is labeled as: Prasad (similarity: 0.79)
Image: Swaroop.jpg, Detected face is labeled as: Swar